### Instalacija biblioteka

In [ ]:
%pip install py_ecc
%pip install ecdsa

### Sabiranje tačaka na eliptičkoj krivoj

In [ ]:
from py_ecc import bn128

In [3]:
# Izračunavanje tačaka na eliptičkoj krivoj
point1 = bn128.multiply(bn128.G1, 123456789)
point2 = bn128.multiply(bn128.G1, 123456789)

# Sabiranje tačaka
new_point = bn128.add(point1, point2)

# Prikaz rezultata
print("Point 1:", point1)
print("Point 2:", point2)
print("New Point after addition:", new_point)


Public Key 1: (9121282642809701931333593728297233225556711250127745709186816755779879923737, 8783642022119951289582979607207867126556038468480503109520224385365741455513)
Public Key 2: (9762719899689846518288039964454931444106783023933610630836848832700633833209, 13418320294483086350796880924221804082812823986598771920845388056528867483634)
New Point after addition: (13941630850939668346535525444231863671048532303622306743285719356065443150089, 7889020351330448120967713518583393111106797395434742462527316530535004742599)


### Pairing

In [4]:
# Izračunavanje javnih ključeva (tačke na eliptičkoj krivoj)
pub_key_g1 = bn128.multiply(bn128.G1, priv_key_1)  # Javna tačka iz G1
pub_key_g2 = bn128.multiply(bn128.G2, priv_key_2)  # Javna tačka iz G2

# Izračunavanje pairing-a
pairing_result = bn128.pairing(pub_key_g2, pub_key_g1)  # Obrni redosled

# Prikaz rezultata
print("Public Key 1 (G1):", pub_key_g1)
print("Public Key 2 (G2):", pub_key_g2)
print("Pairing e(Public Key 1, Public Key 2):", pairing_result)


Public Key 1 (G1): (9121282642809701931333593728297233225556711250127745709186816755779879923737, 8783642022119951289582979607207867126556038468480503109520224385365741455513)
Public Key 2 (G2): ((6290145982231384738074966365038828944845698038674138502655610483714605422009, 801151260611335460165145578655139439644723815179970173493264417463580420976), (7104388331783540487822203877085417454795771769478266307570657875726917971553, 20010759952662045428129942237803352455477642709483925908689586853899117112468))
Pairing e(Public Key 1, Public Key 2): (13155422563560481797803349738873078602077166065460017251298581562538598421049, 19824154330296926943721925684261979459094550948660997753632198356004541076367, 4654204599146452523798251484574286148456113467929161996382402597754669206287, 12596372193435168378859608022090420592376183078071091649065932672326432102872, 16118454153719557091630909783301404957504251829856064066665814395845862171468, 291891640967044307916546625863768596744833098310099560

### Potpisivanje poruka pomoću ECDSA

In [ ]:
from ecdsa import SigningKey, SECP256k1

In [5]:
priv_key = SigningKey.generate(curve=SECP256k1)

pub_key = priv_key.get_verifying_key()

message = b"Verify my signature!"

# Potpisivanje
signature = priv_key.sign(message)

# Verifikacija
is_valid = pub_key.verify(signature, message)
print(f"Signature: {signature.hex()}")
print(f"Signature Valid: {is_valid}")

Signature: a22ad56028ec7ba1b231a36fd79e71002542ed2576f1e5ac1297f51e11fe2bd088beb61e605265cddd94c8021376153b7c115068bf7aaa1542074a5102bc6acf
Signature Valid: True


### BLS potpis

In [ ]:
from hashlib import sha256
from py_ecc.bls import G2ProofOfPossession as bls

In [6]:
priv_key = bls.KeyGen(sha256(b'private_seed').digest())
pub_key = bls.SkToPk(priv_key)
print(f"Public Key: {pub_key.hex()}")

message = b"Message for BLS signing"
signature = bls.Sign(priv_key, message)
print(f"BLS Signature: {signature.hex()}")

# Verifikacija potpisa
is_valid = bls.Verify(pub_key, message, signature)
print(f"Signature Valid: {is_valid}")

Public Key: b674edf842a1cbf25b206ee258db7719ea317d003b01b2197833e3f95d20601cb377908af8a587e83b9ada42c05cb33c
BLS Signature: 9318a676e9c1be363d9ee1890f4a2c734b4f57dc910fa9424aaa973f43efc33b580b2979c7e5b64b0d0ebf540377031f0528e85119a45feff05882d8478b057c6e08891ad714eb8a50a6d3f2ef3efd2a138acab2b1ba890362e22ce941be1031
Signature Valid: True


### Agregirani BLS potpis

In [7]:
priv_key1, priv_key2 = 12345, 67890
pub_key1, pub_key2 = bls.SkToPk(priv_key1), bls.SkToPk(priv_key2)
print(f"Public Key 1: {pub_key1.hex()}")
print(f"Public Key 2: {pub_key2.hex()}")

message = b"Aggregated signature message"
signature1 = bls.Sign(priv_key1, message)
signature2 = bls.Sign(priv_key2, message)
print(f"BLS Signature 1: {signature1.hex()}")
print(f"BLS Signature 2: {signature2.hex()}")

# Agregirani javni ključeva
aggregated_pub_keys = [pub_key1, pub_key2]
print(f"Aggregated Public key: {pub_key2.hex()}")

aggregated_message = [message, message]

# Agregirani potpis
aggregated_signature = bls.Aggregate([signature1, signature2])
print(f"BLS Aggregated Signature: {aggregated_signature.hex()}")

# Verifikacija agregiranog potpisa
is_valid = bls.AggregateVerify(aggregated_pub_keys, aggregated_message, aggregated_signature)
print(f"Aggregated Signature Valid: {is_valid}")

Public Key 1: 8530c1bdc4cd6b1408be0933c4a41ac3513350eef36850b804708e1f338932ce01b655a163344a4500b281c8750c461f
Public Key 2: 85ee0a7d7e181a6894d4c3c6c4581c8d4841ce1dc4bfb3b4bec3f84cc998e4e64e6d2110fc32d35b7f9726221150d9b5
BLS Signature 1: b693b965e9b5f444fa99ee9f686d08ea82f8f1cf0a083db024870f883760e6cabf432f9ea50ae1049a2f93d00ea729860d5cd8713b6ac92a68ea63e650988887a36a5d5f730568133e21319db9400e7b2fc89afd83b1f02388a65fddc803154f
BLS Signature 2: 8948af389737ffdd6e4045569b99660e7fa3b65b27b131947eccdca2086d9aac3cef71d1735ee24a0764bab0fff51fc0197edcf9777708746bfbbeb30e4c2365599fb28e793693c1ad5e624499e320ec3b8cb2528db3235c4766f6c06b7a4b31
Aggregated Public key: 85ee0a7d7e181a6894d4c3c6c4581c8d4841ce1dc4bfb3b4bec3f84cc998e4e64e6d2110fc32d35b7f9726221150d9b5
BLS Aggregated Signature: 951971dc68d94b8a65dcbd0e2187d47c81ad3f97faf44d6d42e22742e0a0a631833db72b621bd8b331b9a9ea1afabb2a0e7d3a39e522eecd7c3ae3af5b7fe16fd0c78945f5716347ed26207a2fbb1848db24ab4452696a3f7c558b78016f0db7
Aggregated Signatur